<a href="https://colab.research.google.com/github/Rukhnuddin786/P3-Spam-Email-Classification-AICTE-Intership/blob/main/storyteller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install cohere gtts pydub nltk spacy python-dotenv soundfile numpy
!python -m spacy download en_core_web_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!pip install cohere gtts pydub nltk spacy python-dotenv soundfile numpy ipywidgets
!python -m spacy download en_core_web_lg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.2 MB/s eta 0:00:00
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.8.0/en_core_web_lg-3.8.0-py3-none-any.whl (400.7 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:

# %%
import os
import cohere
from gtts import gTTS
from pydub import AudioSegment
import nltk
import spacy
from datetime import datetime
import numpy as np
from IPython.display import Audio, display, clear_output
import ipywidgets as widgets
from google.colab import userdata


nltk.download('punkt')


COHERE_API_KEY = userdata.get('COHERE_API_KEY')


class StoryTeller:
    def __init__(self):

        self.co = cohere.Client(COHERE_API_KEY)


        self.nlp = spacy.load('en_core_web_lg')


        self.audio_settings = {
            'speed': 1.0,
            'pitch': 1.0,
            'volume': 1.0
        }

    def generate_story(self, prompt, creativity=0.7, length='medium'):
        """Generate a story based on the given prompt"""
        length_map = {
            'short': 300,
            'medium': 600,
            'long': 900
        }

        try:
            response = self.co.generate(
                model='command',
                prompt=f"Write a creative story based on: {prompt}\n\nStory:",
                max_tokens=length_map.get(length, 600),
                temperature=creativity,
                k=0,
                stop_sequences=[],
                return_likelihoods='NONE'
            )

            story = response.generations[0].text
            return self._post_process_story(story)
        except Exception as e:
            print(f"Error generating story: {e}")
            return None

    def _post_process_story(self, text):
        """Clean and format the generated story"""
        text = text.strip()
        doc = self.nlp(text)
        sentences = [sent.text for sent in doc.sents]
        return ' '.join(sent.capitalize() for sent in sentences)

    def text_to_speech(self, text, output_format='mp3', voice='en'):
        """Convert text to speech with adjustable parameters"""
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"story_{timestamp}"
            temp_file = f"temp_{filename}.mp3"


            tts = gTTS(text=text, lang=voice, slow=False)
            tts.save(temp_file)


            audio = AudioSegment.from_mp3(temp_file)
            audio = self._adjust_audio(audio)


            output_file = f"{filename}.{output_format}"
            audio.export(output_file, format=output_format)
            os.remove(temp_file)

            return output_file
        except Exception as e:
            print(f"Error in TTS conversion: {e}")
            return None

    def _adjust_audio(self, audio_segment):
        """Apply audio adjustments"""

        if self.audio_settings['speed'] != 1.0:
            speed = self.audio_settings['speed']
            sound = audio_segment._spawn(
                audio_segment.raw_data,
                overrides={"frame_rate": int(audio_segment.frame_rate * speed)}
            )
            audio_segment = sound.set_frame_rate(audio_segment.frame_rate)


        if self.audio_settings['volume'] != 1.0:
            change_in_dB = 20 * np.log10(self.audio_settings['volume'])
            audio_segment = audio_segment + change_in_dB

        return audio_segment

    def set_audio_parameters(self, speed=None, pitch=None, volume=None):
        """Update audio settings"""
        if speed is not None:
            self.audio_settings['speed'] = max(0.5, min(2.0, speed))
        if pitch is not None:
            self.audio_settings['pitch'] = max(0.5, min(2.0, pitch))
        if volume is not None:
            self.audio_settings['volume'] = max(0.1, min(2.0, volume))


def create_interface():

    prompt_input = widgets.Textarea(
        value='A young explorer discovers a hidden city in the Amazon rainforest',
        placeholder='Enter your story prompt here...',
        description='Prompt:',
        layout=widgets.Layout(width='80%', height='100px')
    )

    creativity_slider = widgets.FloatSlider(
        value=0.7, min=0.1, max=1.0, step=0.1,
        description='Creativity:', readout_format='.1f')

    length_radio = widgets.RadioButtons(
        options=['short', 'medium', 'long'],
        value='medium',
        description='Length:')

    speed_slider = widgets.FloatSlider(
        value=1.0, min=0.5, max=2.0, step=0.1,
        description='Speed:', readout_format='.1f')

    volume_slider = widgets.FloatSlider(
        value=1.0, min=0.1, max=2.0, step=0.1,
        description='Volume:', readout_format='.1f')

    generate_btn = widgets.Button(
        description='Generate Story & Audio',
        button_style='success',
        layout=widgets.Layout(width='300px'))

    output_area = widgets.Output()
    audio_output = widgets.Output()


    def on_generate_clicked(b):
        with output_area:
            clear_output()
            print("Generating story... (this may take a moment)")


            storyteller = StoryTeller()
            storyteller.set_audio_parameters(
                speed=speed_slider.value,
                volume=volume_slider.value)


            story = storyteller.generate_story(
                prompt_input.value,
                creativity=creativity_slider.value,
                length=length_radio.value)

            if story:
                print("\nGenerated Story:")
                print("-"*50)
                print(story)
                print("-"*50)


                with audio_output:
                    clear_output()
                    print("Generating audio...")
                    audio_file = storyteller.text_to_speech(story)
                    if audio_file:
                        print("Playing your story...")
                        display(Audio(audio_file))
            else:
                print("Failed to generate story.")

    generate_btn.on_click(on_generate_clicked)


    controls = widgets.VBox([
        prompt_input,
        creativity_slider,
        length_radio,
        widgets.HBox([speed_slider, volume_slider]),
        generate_btn
    ])

    display(widgets.VBox([
        widgets.HTML("<h2>Story Generator</h2>"),
        controls,
        output_area,
        audio_output
    ]))


create_interface()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
